In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/stock-index-prediction-both-labels-and-features/NASDAQ.csv
/kaggle/input/stock-index-prediction-both-labels-and-features/DOW30.csv
/kaggle/input/stock-index-prediction-both-labels-and-features/SP500.csv


In [2]:
import numpy as np
import tensorflow.compat.v1 as tf
import matplotlib.pyplot as plt
import seaborn as sb
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
tf.disable_eager_execution()

In [3]:
import pandas as pd
DOW30 = pd.read_csv("../input/stock-index-prediction-both-labels-and-features/DOW30.csv")
NASDAQ = pd.read_csv("../input/stock-index-prediction-both-labels-and-features/NASDAQ.csv")
df = pd.read_csv("../input/stock-index-prediction-both-labels-and-features/SP500.csv")

In [4]:
df.head()

,Date,LABEL,Open,High,Low,Close,Volume,InterestRate,ExchangeRate,VIX,Gold,Oil,TEDSpread,EFFR
0,2008/4/1,0,1326.410034,1370.180054,1326.410034,1370.180054,4.745120e+09,1.77,1.5615,22.68,897.00,100.92,1.30,2.38
1,2008/4/2,1,1369.959961,1377.949951,1361.550049,1367.530029,4.320440e+09,1.72,1.5618,23.43,893.50,104.83,1.31,2.18
2,2008/4/3,1,1365.689941,1375.660034,1358.680054,1369.310059,3.920100e+09,1.70,1.5667,23.21,898.25,103.92,1.35,2.19
3,2008/4/4,1,1369.849976,1380.910034,1362.829956,1370.400024,3.703100e+09,1.67,1.5735,22.45,905.25,106.09,1.40,2.26
4,2008/4/7,0,1373.689941,1386.739990,1369.020020,1372.540039,3.747780e+09,1.62,1.5713,22.42,914.70,108.91,1.28,2.24


In [5]:
data_df = df.drop(['Date', 'LABEL'], axis=1)

In [6]:
data_df.head()

,Open,High,Low,Close,Volume,InterestRate,ExchangeRate,VIX,Gold,Oil,TEDSpread,EFFR
0,1326.410034,1370.180054,1326.410034,1370.180054,4.745120e+09,1.77,1.5615,22.68,897.00,100.92,1.30,2.38
1,1369.959961,1377.949951,1361.550049,1367.530029,4.320440e+09,1.72,1.5618,23.43,893.50,104.83,1.31,2.18
2,1365.689941,1375.660034,1358.680054,1369.310059,3.920100e+09,1.70,1.5667,23.21,898.25,103.92,1.35,2.19
3,1369.849976,1380.910034,1362.829956,1370.400024,3.703100e+09,1.67,1.5735,22.45,905.25,106.09,1.40,2.26
4,1373.689941,1386.739990,1369.020020,1372.540039,3.747780e+09,1.62,1.5713,22.42,914.70,108.91,1.28,2.24


In [7]:
data_df_X = data_df.drop(['Close'], axis=1)[:-1]
data_df_y = data_df['Close']
data_df_y = data_df_y.shift(-1, axis=0)[:-1]
X = data_df_X.values
y = data_df_y.values

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, shuffle=False)
y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

In [9]:
scaler = MinMaxScaler()
X_train_norm = scaler.fit_transform(X_train)
X_test_norm = scaler.fit_transform(X_test)
y_train_norm = scaler.fit_transform(y_train)
y_test_norm = scaler.fit_transform(y_test)

In [10]:
n_input = 11
n_l1 = 8
n_l2 = 8
n_l3 = 8
n_l4 = 8
n_l5 = 8
n_out = 1

In [11]:
X = tf.placeholder(dtype=tf.float32, shape=[None, n_input])
Y = tf.placeholder(dtype=tf.float32, shape=[None, 1])

In [12]:
sigma = 1
weight_initializer = tf.variance_scaling_initializer(mode="fan_avg", distribution="uniform", scale=sigma)
bias_initializer = tf.zeros_initializer()

In [13]:
w1 = tf.Variable(weight_initializer([n_input, n_l1]))
b1 = tf.Variable(bias_initializer([n_l1]))
w2 = tf.Variable(weight_initializer([n_l1, n_l2]))
b2 = tf.Variable(bias_initializer([n_l2]))
w3 = tf.Variable(weight_initializer([n_l2, n_l3]))
b3 = tf.Variable(bias_initializer([n_l3]))
w4 = tf.Variable(weight_initializer([n_l3, n_l4]))
b4 = tf.Variable(bias_initializer([n_l4]))
w5 = tf.Variable(weight_initializer([n_l4, n_l5]))
b5 = tf.Variable(bias_initializer([n_l5]))
w_out = tf.Variable(weight_initializer([n_l5, n_out]))
b_out = tf.Variable(bias_initializer([n_out]))

In [14]:
l1 = tf.nn.relu(tf.add(tf.matmul(X, w1), b1))
l2 = tf.nn.relu(tf.add(tf.matmul(l1, w2), b2))
l3 = tf.nn.relu(tf.add(tf.matmul(l2, w3), b3))
l4 = tf.nn.relu(tf.add(tf.matmul(l3, w4), b4))
l5 = tf.nn.relu(tf.add(tf.matmul(l4, w5), b5))
out = tf.transpose(tf.add(tf.matmul(l5, w_out), b_out))

In [15]:
mse = tf.reduce_mean(tf.squared_difference(out, Y))
opt = tf.train.AdamOptimizer().minimize(mse)

In [16]:
test_session = tf.Session(config=tf.ConfigProto(log_device_placement=True))

Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0



In [17]:
test_session.run(tf.global_variables_initializer())

In [18]:
batch_size = 89
for e in range(300):
    for i in range(len(y_train)):
        start = i * batch_size
        batch_X = X_train_norm[start : start + batch_size]
        batch_y = y_train_norm[start : start + batch_size]
        test_session.run(opt, feed_dict={X: batch_X, Y: batch_y})

In [20]:
test_session.run(mse, feed_dict={X: X_test_norm, Y: y_test_norm})

0.16878507

In [ ]:
predictions = test_session.run(out, feed_dict={X: X_test_norm})

In [19]:
predictions_denorm = scaler.inverse_transform(predictions)

NameError: name 'predictions' is not defined

In [ ]:
plt.plot(range(len(y_test)), y_test, color='b')
plt.plot(range(len(y_test)), predictions_denorm.flatten(), color='r')
plt.show()